In [1]:
import subprocess
import sys

In [2]:
def install_package(package):
    """Installs the package using pip"""
    subprocess.check_call([sys.executable, '-m', 'pip','install',package])

In [3]:
def check_and_install_dependencies(packages):
    """Checks if packages are installed, installs them if not"""
    for package in packages:
        try:
            __import__(package.split('==')[0])
            print(f"{package} is alreday installed")
        except ImportError:
            print(f'{package} not found. Installing...')
            install_package(package)
            print(f'{package} has been installed.')

In [4]:
import ctypes
import serial
import time
import serial.tools.list_ports
from datetime import datetime
from collections import Counter

In [5]:
check_and_install_dependencies(['pyserial==3.5','serial==0.0.97','psycopg2-binary'])

pyserial==3.5 not found. Installing...



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


pyserial==3.5 has been installed.
serial==0.0.97 is alreday installed
psycopg2-binary not found. Installing...
psycopg2-binary has been installed.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [6]:
data_original=open('data.csv','r').readlines()

In [22]:
# Function to extract selected columns
def extract_columns(data, selected_columns,delimiter=';'):
    '''
    Extracts the intended data given the indeces in selected_columns
    Determines which is the current state type.
    '''
    columns = data.split(delimiter)  
    result = [str(int(columns[i])/1000) if i==7 else columns[i] for i in selected_columns]
    if result[3]=='1':
        estado='charging'
    elif result [3]=='2':
        estado='discharging'
    elif result [3]=='4':
        estado='rest'
    elif result[3]=='6':
        estado='finished'
    result[3]=estado
    return delimiter.join(result),estado #return the data as string

def save_file(estado,bateria,capacidad,ciclo,data):
    '''
    Function for saving the data file (.csv)
    By data analisis, each current state is compared to the previous four (4) states. 
    When the 5 states changes, then it's certain the battery power state has changed
    '''
    estados_pasados.append(estado)
    #determine the correct battery state (charging, resting, discharging) to save data to
    if len(estados_pasados)>4:
        if all(x==estado for x in estados_pasados):
            file_name=f"{bateria}{estado}_{capacidad}_{ciclo}.csv"
        else:
            most_common_elem, count = Counter(estados_pasados).most_common(1)[0]
            if most_common_elem=="charging" and count==3 and estado=='charging':
                ciclo+=1
            file_name=f"{bateria}{most_common_elem}_{capacidad}_{ciclo}.csv"
        if not(file_name in dict_data):
            dict_data[file_name]=[]
            state_file = open(file_name, "w")
            state_file.write('date;system_hour;cycle_time;battery_state;voltage[V];current[mA];capacity[mAh]'+'\n')#setting column titles
            state_file.flush()
        dict_data[file_name].append(data)
        estados_pasados.pop(0)
        
        #writing data to the specific file
        try:
            state_file = open(file_name, "a")
        except:
            print("book is already open")
        finally:
            state_file.write(data+'\n')
            state_file.flush()
    return ciclo

In [23]:
log_to_file=True
estados_pasados=[]
dict_data={}
columns_to_extract=[0,1,2,4,7,8,17]#date, system_time,cycle_time, battery_state(charing/resting/discharging),voltage(V),current(mA),capacity(mAh)
bateria='1'
capacidad='1000'
ciclo=1
base_time=datetime.strptime("08:51:02","%H:%M:%S")

if log_to_file:
    log_file = open(f"data_original_{bateria}_{capacidad}_{ciclo}.csv", "w")
    log_file.write('date;system_hour;cicle_time;battery_state;voltage[V];current[mA];capacity[mAh]'+'\n')#setting column titles
    log_file.flush()
else:
    log_file = None
                
for a in data_original[:-1]:
    #adding cycle time (stopwatch like)
    aux=a.split(';')
    time1=datetime.strptime(aux[1],"%H:%M:%S")
    diff=str(abs(time1-base_time))
    aux.insert(2,diff)
    a=';'.join(aux)
    data,estado=extract_columns(a,columns_to_extract)
    if estados_pasados==['finished','finished','finished','finished']:
        print("finished cicles \nclosing program")
        print(a.split(';')[0])
        break
        
    if log_to_file:
        #original data
        if log_file:
            log_file.write(data + '\n')
            log_file.flush()
            ciclo=save_file(estado,bateria,capacidad,ciclo,data)   

finished cicles 
closing program
2025-09-23


In [24]:
#read from CSV to DB

In [25]:
import os
import csv
from pathlib import Path
import re
txt_files = [f for f in os.listdir(os.getcwd()) if f.endswith('.csv') and os.path.isfile(os.path.join(os.getcwd(), f))]

In [26]:
#getting the correct row. Not always the first row has the column titles. This may be found in the first five (5) rows
def get_column_title(file):
    csv_path = Path(file)
    if not csv_path.exists():
        raise FileNotFoundError(f"{csv_file_path} not found.")
    with csv_path.open('r', newline='', encoding='utf-8') as f:
        aux=False
        count=0
        while not(aux):
            line=f.readline()
            if line.count(';')>=3:
                aux=True
    #order the columns (date, time, voltage, current, capacity)
        order=order_columns(line.strip())
        headings=[a for a in order.keys()]
        if 'date' not in order:
            try:
                date= re.search(r'\d{4}-\d{2}-\d{2}', file)
                headings.insert(0,'date')
            except ValueError:
                pass
    #extract the data in the desired order
        aux=0
        print(headings)
        while aux<10: #extracting the remaining data
            line=f.readline().strip()
            data=line.split(';')
            db_data=[data[a] for a in order.values()]
            if 'date' not in order:
                db_data.insert(0,date.group(0))
            print(db_data)
            aux+=1

In [27]:
def order_columns(header_row):
    a=header_row.lower().split(';')
    expected_columns=['date','time','voltage','current','capacity']
    file_column_order={} #key=battery parameter, value=column number
    for aux in expected_columns:
     	index=0
     	while index<len(a):
     		if aux in a[index]:
     			file_column_order[aux]=index
     			break
     		index+=1
    return file_column_order

In [30]:
for a in txt_files[2:7]:
    b=get_column_title(a)
    print(f'file {a}, extracted column: {b}')
    print()

['date', 'time', 'voltage', 'current', 'capacity']
['2025-09-23', '15:54:47', '4.197', '6', '0']
['2025-09-23', '15:54:49', '4.197', '6', '0']
['2025-09-23', '15:54:51', '4.197', '6', '0']
['2025-09-23', '15:54:53', '4.197', '6', '0']
['2025-09-23', '15:54:55', '4.197', '6', '0']
['2025-09-23', '15:54:58', '4.196', '6', '0']
['2025-09-23', '15:54:59', '4.197', '6', '0']
['2025-09-23', '15:55:01', '4.196', '6', '0']
['2025-09-23', '15:55:03', '4.201', '6', '0']
['2025-09-23', '15:55:05', '4.196', '6', '0']
file 1charging_1_3.csv, extracted column: None

['date', 'time', 'voltage', 'current', 'capacity']
['2025-09-23', '7:32:41', '3.285', '50', '0']
['2025-09-23', '7:32:43', '3.284', '50', '0']
['2025-09-23', '7:32:45', '3.3', '49', '1']
['2025-09-23', '7:32:47', '3.31', '49', '1']
['2025-09-23', '7:32:49', '3.32', '49', '1']
['2025-09-23', '7:32:51', '3.311', '49', '1']
['2025-09-23', '7:32:53', '3.311', '49', '2']
['2025-09-23', '7:32:55', '3.312', '50', '2']
['2025-09-23', '7:32:57', 